In [2]:
using Pkg
Pkg.activate(".")
using SymPy
using LinearAlgebra

  Activating project at `c:\Users\tim\Documents\2025FallExamples\11-19`


In [3]:
Dot(a, b) = sum(a .* b)
struct quaternion
    r
    v
end
function quaternion(a0,a1,a2,a3)
    return quaternion(a0, [a1,a2,a3])
end
function norm(q::quaternion)
    return sqrt( q.r^2 + Dot(q.v,q.v))
end
function inv(q::quaternion)
    return quaternion(q.r/norm(q)^2, -q.v/norm(q)^2 )
end
function Base.:+(a::quaternion, b::quaternion)
    return quaternion(a.r + b.r, a.v + b.v)
end
function Base.:*(a::quaternion, b::quaternion)
    return quaternion(a.r * b.r - Dot(a.v, b.v), a.r * b.v + b.r * a.v + cross(a.v, b.v))
end
function Base.:*(a::Real, b::quaternion)
    return quaternion(a*b.r, a*b.v)
end
function Base.:*(b::quaternion, a::Real)
    return quaternion(a * b.r, a * b.v)
end
function Base.:/(b::quaternion, a::Real)
    return quaternion(b.r/a, b.v/a)
end

tilde(x) = [0 -x[3] x[2]; x[3] 0 -x[1]; -x[2] x[1] 0]
C(q) = q.v * transpose(q.v) + q.r^2 * I - 2 * q.r * tilde(q.v) + tilde(q.v)^2


C (generic function with 1 method)

In [4]:
q = quaternion(1,[2,3,4])

quaternion(1, [2, 3, 4])

In [5]:
q1 = quaternion(1,2,3,4)
q1 = q1/norm(q1)

q2 = quaternion(-2,0,4,5)
q2 = q2/norm(q2)

quaternion(-0.29814239699997197, [0.0, 0.5962847939999439, 0.7453559924999299])

In [6]:
q3 = q1*q2

quaternion(-0.9253627917180894, [-0.1360827634879544, -0.3265986323710904, 0.13608276348795434])

In [7]:
C(q3) - C(q2)*C(q1)

3×3 Matrix{Float64}:
 1.11022e-16  -5.55112e-17  0.0
 0.0           0.0          1.11022e-16
 0.0          -2.22045e-16  0.0

In [8]:
R1(θ) = [1 0 0;
    0 cos(θ) sin(θ);
    0 -sin(θ) cos(θ)]
R2(θ) = [cos(θ) 0 -sin(θ);
    0 1 0;
    sin(θ) 0 cos(θ)]
R3(θ) = [cos(θ) sin(θ) 0;
    -sin(θ) cos(θ) 0;
    0 0 1]
C_321(ψ,θ,ϕ) = R1(ϕ)*R2(θ)*R3(ψ)

C_321 (generic function with 1 method)

In [10]:
function CtoEulerParameters(C)

    trC = tr(C)

    bsq = 1/4*[ 1 + trC, 
        1 + 2*C[1,1] - trC,
        1 + 2*C[2,2] - trC,
        1 + 2*C[3,3] - trC ]
    i = argmax(bsq)

    if i == 1
        β0 = sqrt(bsq[1])
        
        β1 = 1/4*(C[2,3]-C[3,2])/β0
        β2 = 1/4*(C[3,1]-C[1,3])/β0
        β3 = 1/4*(C[1,2]-C[2,1])/β0

    elseif i == 2
        β1 = sqrt(bsq[2])
        
        β0 = 1 / 4 * (C[2,3] - C[3,2]) / β1
        β2 = 1 / 4 * (C[1,2] + C[2,1]) / β1
        β3 = 1 / 4 * (C[3,1] + C[1,3]) / β1

    elseif i == 3
        β2 = sqrt(bsq[3])

        β0 = 1 / 4 * (C[3,1] - C[1,3]) / β2
        β1 = 1 / 4 * (C[1,2] + C[2,1]) / β2
        β3 = 1 / 4 * (C[2,3] + C[3,2]) / β2

    elseif i == 4
        β3 = sqrt(bsq[4])

        β0 = 1 / 4 * (C[1,2] - C[2,1]) / β3
        β1 = 1 / 4 * (C[3,1] + C[1,3]) / β3
        β2 = 1 / 4 * (C[2,3] + C[3,2]) / β3

    end

    return quaternion(β0,β1,β2,β3)

end

CtoEulerParameters (generic function with 1 method)

In [27]:
CtoEulerParameters( C(q3) )

quaternion(0.9253627917180894, [0.1360827634879544, 0.3265986323710904, -0.13608276348795434])

In [28]:
C(-1*q3) - C(q3)

3×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0
 0.0  0.0  0.0